In [1]:
# Install necessary libraries
!pip install transformers datasets sentence-transformers faiss-cpu fastapi nest-asyncio uvicorn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 14.8 MB/s eta 0:00:00


In [2]:
from google.colab import files

uploaded = files.upload()

Saving customer_support_tickets.csv to customer_support_tickets.csv


In [6]:
!pip install huggingface_hub
from huggingface_hub import login

login(token="you token")


In [8]:
import pandas as pd

# Load the data into a pandas DataFrame
df = pd.read_csv('customer_support_tickets.csv')

# Display the first few rows to confirm it loaded correctly
display(df.head())

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [11]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for each ticket
embeddings = embed_model.encode(df['Ticket Description'].tolist(), convert_to_numpy=True)

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

print("FAISS index created with", index.ntotal, "vectors")

FAISS index created with 8469 vectors


In [12]:
def retrieve_ticket_context(query, top_k=3):
    """
    Retrieve top_k most similar tickets from FAISS index for a given query
    """
    query_vec = embed_model.encode([query], convert_to_numpy=True)
    distances, indices = index.search(query_vec, top_k)
    results = df.iloc[indices[0]]['Ticket Description'].tolist()
    return results

# Test the retrieval
query = "I cannot login to my account"
context = retrieve_ticket_context(query)
print("Retrieved Context:", context)


Retrieved Context: ["I'm facing issues logging into my {product_purchased} account. It says my account is locked. What should I do to unlock it?\n\nMy name is,\n\n$My Account ID\n\nAnd the password I gave This problem started occurring after the recent software update. I haven't made any other changes to the device.", "I'm facing issues logging into my {product_purchased} account. It says my account is locked. What should I do to unlock it? I'm worried that the issue might be hardware-related and might require repair or replacement.", "I'm facing issues logging into my {product_purchased} account. It says my account is locked. What should I do to unlock it? I'm worried that the issue might be hardware-related and might require repair or replacement."]


In [13]:
from transformers import pipeline

# Load text-generation model (runs on Colab GPU)
generator = pipeline("text-generation", model="google/flan-t5-small", device=0)

def generate_response(ticket):
    # Retrieve context
    context = retrieve_ticket_context(ticket)

    # Create prompt with context
    prompt = f"You are a helpful customer support assistant.\nTicket: {ticket}\nContext: {context}\nProvide a helpful response:"

    # Generate response
    response = generator(prompt, max_length=150, do_sample=True)
    return response[0]['generated_text']

# Test it
ticket = "I cannot login to my account"
response = generate_response(ticket)
print("Generated Response:\n", response)


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForC

Generated Response:
 You are a helpful customer support assistant.
Ticket: I cannot login to my account
Context: ["I'm facing issues logging into my {product_purchased} account. It says my account is locked. What should I do to unlock it?\n\nMy name is,\n\n$My Account ID\n\nAnd the password I gave This problem started occurring after the recent software update. I haven't made any other changes to the device.", "I'm facing issues logging into my {product_purchased} account. It says my account is locked. What should I do to unlock it? I'm worried that the issue might be hardware-related and might require repair or replacement.", "I'm facing issues logging into my {product_purchased} account. It says my account is locked. What should I do to unlock it? I'm worried that the issue might be hardware-related and might require repair or replacement."]
Provide a helpful response:


In [14]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "gpt2"  # beginner-friendly, small
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

# Test
response = generator("I cannot login to my account. Help me solve it.", max_new_tokens=100)
print(response[0]['generated_text'])

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I cannot login to my account. Help me solve it.

Click here to report a bug


In [15]:
context_short = " ".join([c[:200] for c in context])  # first 200 chars of each retrieved ticket
prompt = f"You are a customer support assistant.\nTicket: I cannot login to my account\nContext: {context_short}\nProvide a concise helpful response:"
response = generator(prompt, max_new_tokens=100)
print(response[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a customer support assistant.
Ticket: I cannot login to my account
Context: I'm facing issues logging into my {product_purchased} account. It says my account is locked. What should I do to unlock it?

My name is,

$My Account ID

And the password I gave This problem started o I'm facing issues logging into my {product_purchased} account. It says my account is locked. What should I do to unlock it? I'm worried that the issue might be hardware-related and might require repai I'm facing issues logging into my {product_purchased} account. It says my account is locked. What should I do to unlock it? I'm worried that the issue might be hardware-related and might require repai
Provide a concise helpful response:

"I need to verify that your account is locked. Please contact us if your account is locked. I'll get back to you."

Helpful:

You can use your account as a backup account.

You can use your account as a backup account. You can use it as a user account.

You can use it as a us